<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise we are using high level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

    1. Read the code.

    2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

    3. Add the option to use SGD with momentum instead of ADAM.

    4. Experiment with different learning rates, plot the learning curves for different
    learning rates for both ADAM and SGD with momentum.

    5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
    Note that this requires creating a list of layers as an atribute of the Net class,
    and one can't use a standard python list containing nn.Modules (why?).
    Check torch.nn.ModuleList.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import plotly.graph_objects as go
import numpy as np

In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval, should_print=True):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if should_print:
          if batch_idx % log_interval == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))
            
    return train_loss / len(train_loader.dataset)


def test(model, device, test_loader, should_print=True):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if should_print:
      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          100. * correct / len(test_loader.dataset)))
    
    return test_loss



In [4]:
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
use_cuda = False
seed = 1
log_interval = 10

In [5]:
use_cuda = not use_cuda and torch.cuda.is_available()

torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [6]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313259
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.732760
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.582762
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.481087
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.334523
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.346265
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.229349
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.191099
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.224889
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.244065
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.236049
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.241033
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.194209
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.302404
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.203934
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.192445
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.173173
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.140804
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.356528
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.130329
Train Epoc

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298198
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.181310
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.797345
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.272565
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.658750
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.662068
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.419146
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.344355
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.430322
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.358123
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.325542
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.412715
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.412903
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.365308
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.332432
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.355307
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.297482
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.253185
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.426848
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.193856
Train Epoc

In [27]:
from functools import partial

model = Net().to(device)

epochs = 5
optimizers = {
    'Adam': optim.Adam,
    'SGD': partial(optim.SGD, momentum=0.9, nesterov=True)
}
learning_rates = [1e-5, 1e-4, 1e-3,1e-2,1e-1,1]

for (optimizer_name, optimizer_class) in optimizers.items():
  fig_train = go.Figure()
  fig_val = go.Figure()

  for lr in learning_rates:
    optimizer = optimizer_class(model.parameters(), lr=lr)
    train_losses = []
    test_losses = []
    for epoch in range(1, epochs + 1):
        train_loss = train(model, device, train_loader, optimizer, epoch, log_interval=log_interval, should_print=False)
        test_loss = test(model, device, test_loader, should_print=False)

        train_losses.append(train_loss)
        test_losses.append(test_loss)

        
    fig_train.add_trace(go.Scatter(
        x=np.arange(1, epochs+1),
        y=train_losses,
        name=f"Train lr={lr}"
    ))

    fig_val.add_trace(go.Scatter(
        x=np.arange(1, epochs+1),
        y=test_losses,
        name=f"Test lr={lr}"
    ))
    

  fig_train.update_layout(
      title=f"{optimizer_name} losses train",
      xaxis_title="Epochs",
      yaxis_title="Loss",
      legend_title="Learning rate",
      font=dict(
          family="Courier New, monospace",
          size=18,
          color="RebeccaPurple"
      )
  )
  fig_train.show()

  fig_val.update_layout(
      title=f"{optimizer_name} losses val",
      xaxis_title="Epochs",
      yaxis_title="Loss",
      legend_title="Learning rate",
      font=dict(
          family="Courier New, monospace",
          size=18,
          color="RebeccaPurple"
      )
  )
  fig_val.show()


In [36]:
class ExtendedNet(Net):
    def __init__(self, hidden_layers_sizes=[128, 128]):
        super().__init__()

        weights_shapes = zip([784]+hidden_layers_sizes, hidden_layers_sizes+[10])
        self.linears = nn.ModuleList(nn.Linear(n_in, n_out) for n_in, n_out in weights_shapes)

        print(self.linears)

    def forward(self, x):
        x = torch.flatten(x, 1)
        
        for layer_index, fc in enumerate(self.linears):
          x = fc(x)
          if layer_index < len(self.linears)-1:
            x = F.relu(x)

        output = F.log_softmax(x, dim=1)
        return output

model = ExtendedNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

ModuleList(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): Linear(in_features=128, out_features=128, bias=True)
  (2): Linear(in_features=128, out_features=10, bias=True)
)
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302301
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.616348
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.460033
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.270302
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.266026
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.244591
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.278567
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.192516
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.173026
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.158278
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.130462
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.215672
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.136169
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.153167
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.129931
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.120950
Train Epoch: 1 [4